In [ ]:
#Run cell to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import necessary packages

#our workhorses
import numpy as np
import pandas as pd
import scipy

#to visualize
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#style params for figures
sns.set(font_scale = 2)
plt.style.use('seaborn-white')
plt.rc("axes", labelweight="bold")
from IPython.display import display, HTML

#to load files
import os
import sys
import h5py

#append repo folder to search path
sys.path.append('/content/drive/MyDrive/limb-position-EMG-Repo/')
from utils import *



In [ ]:
#define where the data files are located
data_folder = '/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/'
results_folder = '/content/drive/MyDrive/limb-position-EMG-Repo/results_data/single_subject_training/log_reg/'

nsubjects = 36


# User-defined parameters
lo_freq = 20 #lower bound of bandpass filter
hi_freq = 450 #upper bound of bandpass filter

win_size = 100 #define window size over which to compute time-domain features
step = win_size #keeping this parameter in case we want to re-run later with some overlap

nreps = 10
exclude = [0,7]#labels to exclude

#for RNN training
verbose = 0
epochs = 40
batch_size = 2

for subject_id in range(1,nsubjects+1):
    
    subject_folder = os.path.join(data_folder,'%02d'%(subject_id))
    print('=======================')
    print(subject_folder)

    # Process data and get features 
    #get features across segments and corresponding info
    feature_matrix_sub, target_labels_sub, window_tstamps_sub, \
    block_labels_sub, series_labels_sub = get_subject_data_for_classification(subject_folder, lo_freq, hi_freq, \
                                                                    win_size, step)
    np.random.seed(1)#for reproducibility
    results_df = []#initialize empty array for dataframes
    for rep in range(nreps):
        print('Subject %d|Rep %d'%(subject_id, rep+1))

        print('True Data')
        train_f1, test_f1 = within_subject_log_reg_performance(feature_matrix_sub, target_labels_sub, series_labels_sub, exclude,\
                                                                            verbose = 0, epochs = epochs, batch_size = batch_size, permute = False)
        results_df.append(pd.DataFrame({'F1_score':train_f1,\
                            'Fold':np.arange(train_f1.size)+1,\
                            'Rep':[rep+1 for x in range(train_f1.size)],\
                            'Type':['Train' for x in range(train_f1.size)],\
                            'Shuffled':[False for x in range(train_f1.size)],\
                            'Subject':[subject_id for x in range(train_f1.size)],\
                            }))
        results_df.append(pd.DataFrame({'F1_score':test_f1,\
                            'Fold':np.arange(test_f1.size)+1,\
                            'Rep':[rep+1 for x in range(test_f1.size)],\
                            'Type':['Test' for x in range(test_f1.size)],\
                            'Shuffled':[False for x in range(test_f1.size)],\
                            'Subject':[subject_id for x in range(test_f1.size)],\
                            }))
        
        print('Permuted Data')
        train_f1_perm, test_f1_perm = within_subject_log_reg_performance(feature_matrix_sub, target_labels_sub, series_labels_sub, exclude,\
                                                                            verbose = 0, epochs = epochs, batch_size = batch_size, permute = True)
        results_df.append(pd.DataFrame({'F1_score':train_f1_perm,\
                            'Fold':np.arange(train_f1_perm.size)+1,\
                            'Rep':[rep+1 for x in range(train_f1_perm.size)],\
                            'Type':['Train' for x in range(train_f1_perm.size)],\
                            'Shuffled':[True for x in range(train_f1_perm.size)],\
                            'Subject':[subject_id for x in range(train_f1_perm.size)],\
                            }))
        results_df.append(pd.DataFrame({'F1_score':test_f1_perm,\
                            'Fold':np.arange(test_f1_perm.size)+1,\
                            'Rep':[rep+1 for x in range(test_f1_perm.size)],\
                            'Type':['Test' for x in range(test_f1_perm.size)],\
                            'Shuffled':[True for x in range(test_f1_perm.size)],\
                            'Subject':[subject_id for x in range(test_f1_perm.size)],\
                            }))
    results_df = pd.concat(results_df, axis = 0)
    #save results to file
    results_fn = 'subject_%02d_within_subject_results.h5'%(subject_id)
    results_df.to_hdf(os.path.join(results_folder,results_fn), key='results_df', mode='w')


Streaming output truncated to the last 5000 lines.
Evaluate Model
Subject 4|Rep 2
True Data
Split Count: 1
Training Model
Evaluate Model
Split Count: 2
Training Model
Evaluate Model
Permuted Data
Split Count: 1
Training Model
Evaluate Model
Split Count: 2
Training Model
Evaluate Model
Subject 4|Rep 3
True Data
Split Count: 1
Training Model
Evaluate Model
Split Count: 2
Training Model
Evaluate Model
Permuted Data
Split Count: 1
Training Model
Evaluate Model
Split Count: 2
Training Model
Evaluate Model
Subject 4|Rep 4
True Data
Split Count: 1
Training Model
Evaluate Model
Split Count: 2
Training Model
Evaluate Model
Permuted Data
Split Count: 1
Training Model
Evaluate Model
Split Count: 2
Training Model
Evaluate Model
Subject 4|Rep 5
True Data
Split Count: 1
Training Model
Evaluate Model
Split Count: 2
Training Model
Evaluate Model
Permuted Data
Split Count: 1
Training Model
Evaluate Model
Split Count: 2
Training Model
Evaluate Model
Subject 4|Rep 6
True Data
Split Count: 1
Training Mode

In [ ]:


def within_subject_log_reg_performance(X, Y, series_labels, exclude,  verbose = 0, epochs = 40, batch_size = 2, mv = False, permute = False):
    """
    Train and test performance of a logisitc regression model within the same subject
    """

    #initialize object for k-fold cross-validation
    n_splits = np.unique(series_labels).size
    kf = KFold(n_splits=n_splits,shuffle = True)
    #initialize empty arrays
    train_f1_scores = np.empty((n_splits,))
    test_f1_scores = np.empty((n_splits,))

    for split_count, (series_train, series_test) in enumerate(kf.split(np.unique(series_labels))):
        print('Split Count: %i'% (split_count+1))
        #get train and test idxs
        train_idxs = np.where(series_labels==series_train)[0]
        test_idxs = np.where(series_labels==series_test)[0]
        #get training data cubes
        X_train_cube, Y_train_cube, scaler = prepare_data_for_log_reg(X,Y, train_idxs, exclude, train = True)
        if permute:
            perm_idxs = np.random.permutation(np.arange(Y_train_cube.shape[0]))
            Y_train_cube = Y_train_cube[perm_idxs,:]

        n_features, n_outputs = X_train_cube.shape[1], Y_train_cube.shape[1]

        #setting timestep dimension to None 
        model = get_log_reg_model((n_features,),n_outputs)
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary

        print('Training Model')
        # fit network
        history = model.fit(X_train_cube, Y_train_cube, epochs=epochs, batch_size=batch_size, verbose=verbose)

        # # evaluate trained network
        print('Evaluate Model')
        

        if mv:
            # get testing data cubes
            X_test_cube, Y_test_cube, scaler = prepare_data_for_log_reg(X,Y, train_idxs, [], train = False, scaler = scaler)
            y_pred = get_mv_preds(X_test_cube, model, n_votes= 5)+1
            y_true = np.squeeze(np.argmax(Y_test_cube,1))
            include_idxs = np.where(np.isin(y_true,exclude, invert = True))[0]
            y_true = y_true[include_idxs]
            y_pred = y_pred[include_idxs]
            train_f1 = f1_score(y_true,y_pred,average = 'weighted')

            # get testing data cubes
            X_test_cube, Y_test_cube, scaler = prepare_data_for_log_reg(X,Y, test_idxs, [], train = False, scaler = scaler)
            y_pred = get_mv_preds(X_test_cube, model, n_votes= 5)+1
            y_true = np.squeeze(np.argmax(Y_test_cube,1))
            include_idxs = np.where(np.isin(y_true,exclude, invert = True))[0]
            y_true = y_true[include_idxs]
            y_pred = y_pred[include_idxs]
            test_f1 = f1_score(y_true,y_pred,average = 'weighted')
        else:
            #get score for training data
            train_f1 = get_log_reg_f1(X_train_cube, Y_train_cube, model)
            # get testing data cubes
            X_test_cube, Y_test_cube, scaler = prepare_data_for_log_reg(X,Y, test_idxs, exclude, train = False, scaler = scaler)
            #get score for testing data
            test_f1 = get_log_reg_f1(X_test_cube, Y_test_cube, model)
        #put scores in array
        train_f1_scores[split_count] = train_f1
        test_f1_scores[split_count] = test_f1

    return train_f1_scores, test_f1_scores

In [ ]:
n_splits = np.unique(series_labels).size
X = feature_matrix_sub.copy()
Y = target_labels_sub.copy()
exclude =[0,7]

#for RNN training
verbose = 0
epochs = 40
batch_size = 2






    

Split Count: 1
Split Count: 2


Training Model


(593, 7)